In [1]:
import networkx as nx
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

In [ ]:
#################################################################################
############################### Helper Functions  ###############################
#################################################################################

'''
    Global Variables
    N      ::  Number of nodes
    label  ::  Attribute name of influence
    time   ::  Attribute name of time
'''

def set_time(G, value, node=None):
    '''
        Set the time of an individual node in a network G 
        to value or set the time of all nodes to value.
        G      ::  a networkx graph
        node   ::  a reference to a node in G
        value  ::  a non-negative integer
    '''
    if node:
        G.nodes[node][time] = value
    else:
        time_attrib = {i : value for i in range(N)}
        nx.set_node_attributes(G,time_attrib, time)


def set_influence(G, value, node=None):
    '''
        Set the influence of an individual node in a network G 
        to value or set the influence of all nodes to value.
        G      ::  a networkx graph
        node   ::  a reference to a node in G
        value  ::  an integer 0 or 1
    '''
    if node:
        G.nodes[node][label] = value
    else:
        influence_attrib = { i : value for i in range(N) }
        nx.set_node_attributes(G,influence_attrib, label)
        
def get_is_influenced(G, node):
    '''
        Returns if node in G is influenced.
    '''
    return G.nodes[node][label]
        
def get_number_influenced(G):
    '''
        Get the number of influenced nodes.
    '''
    return sum(nx.get_node_attributes(G, label).values())

def check_can_influence(G, node, q):
    '''
        Determines whether a node can be influenced by
        its neighbours. (Threshold Check)
    '''
    if get_is_influenced(G, node) == 1:
        return False
    
    friends = list(G.neighbors(node))
    num_friends = len(friends)
    
    if num_friends == 0:
        return False
    
    num_influenced = sum([1 for friend in friends if G.nodes[friend][label] == 1])
    
    if num_influenced/num_friends > q:
        return True
    return False

#################################################################################
########################## Simulation Helper Functions ##########################
#################################################################################

def spread_influence(G, current_node, phi):
    '''
        Recursive function to spread influence 
        from current_node in G.
    '''
    
    ## Not interesting
    if get_is_influenced(G, current_node) == 0 and not check_can_influence(G, current_node, phi):
        return
    else:
        set_influence(G, 1, current_node)
        ## Find uninfluenced friends
        friends = list(G.neighbors(current_node))
        targets = [friend for friend in friends if G.nodes[friend][label] == 0]
        
        for friend in targets:
            spread_influence(G, friend, phi)
        
def get_vulnerable(G, phi):
    '''
        Check for vulnerable nodes.
        Return list of vulnerable nodes.
    '''
    vulnerable = []
    for node in G.nodes():
        if check_can_influence(G, node, phi):
            vulnerable.append(node)
    return vulnerable
        
def simulate_spread(G, nodes, phi):
    '''
        Simulates the spread of influence starting from each 
        node in nodes and returns a list containing the 
        number of influenced from starting at each node.
    '''
    S = []
    for node in nodes:
        G_tmp = G.copy()
        set_influence(G_tmp, 1, node)
        spread_influence(G_tmp, node, phi)
        
        vulnerable = get_vulnerable(G_tmp, phi)

        while len(vulnerable) != 0:
            for vul in vulnerable:
                spread_influence(G_tmp, vul, phi)
            vulnerable = get_vulnerable(G_tmp, phi)

        S.append(get_number_influenced(G_tmp))
        
    return S

In [ ]:
def 

In [ ]:
def run_simulation(probability):
    G = nx.erdos_renyi_graph(N, probability, directed)
    set_influence(G, 0)
    ## Retrieve influential nodes - top q% and non-influential nodes
    degree_ordered_nodes = sorted(list(G.nodes()), key=lambda x: G.degree(x), reverse=True)
    influential_nodes = degree_ordered_nodes[:int(q*N)]
    normal_nodes = degree_ordered_nodes[int(q*N):]
    ## Simulation
    influential = simulate_spread(G, influential_nodes, phi)
    normal = simulate_spread(G, normal_nodes, phi)
    ## Store results
    S_influential = np.mean(influential)
    S_normal = (np.mean(normal))
    
    return [S_influential, S_normal]

In [20]:
lt = nx.erdos_renyi_graph

In [22]:
def run_simulation_RG(p):
    G = nx.erdos_renyi_graph(N, p)
    

## Graph Options
Possibl

    watts_strogatz_graph  - Small World Property/Short Average Path and High Clustering
    barabasi_albert_graph - Scale-Free Network/Power Law Degree Distribution
    erdos_renyi_graph     - Random Graph/ Poisson/Binomial Degree Distribution
    

In [23]:
#################################################################################
##################################  Paramters  ##################################
#################################################################################


N = 100
N_avg = 10
label = 'is_influenced'
time = 'time'

In [2]:
?nx.barabasi_albert_graph

Signature: nx.barabasi_albert_graph(n, m, seed=None)
Docstring:
Returns a random graph according to the Barabási–Albert preferential
attachment model.

A graph of $n$ nodes is grown by attaching new nodes each with $m$
edges that are preferentially attached to existing nodes with high degree.

Parameters
----------
n : int
    Number of nodes
m : int
    Number of edges to attach from a new node to existing nodes
seed : integer, random_state, or None (default)
    Indicator of random number generation state.
    See :ref:`Randomness<randomness>`.

Returns
-------
G : Graph

Raises
------
NetworkXError
    If `m` does not satisfy ``1 <= m < n``.

References
----------
.. [1] A. L. Barabási and R. Albert "Emergence of scaling in
   random networks", Science 286, pp 509-512, 1999.
File:      ~/anaconda3/lib/python3.7/site-packages/networkx/generators/random_graphs.py
Type:      function


In [4]:
G = nx.barabasi_albert_graph(N, N_avg)

In [10]:
G = nx.erdos_renyi_graph(N, 0.1, directed=True)

In [11]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 100
Number of edges: 967
Average in degree:   9.6700
Average out degree:   9.6700


In [18]:
list(G.neighbors(23))

[1, 12, 14, 19, 24, 66, 86, 93]

In [17]:
G.in_edges(23)

InEdgeDataView([(14, 23), (36, 23), (54, 23), (76, 23), (78, 23), (83, 23)])